In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import robust_scale
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import scipy.stats as stats
import statistics

In [2]:
df=pd.read_csv('worker_matrix.csv')

#### If we calculate correlation vs work duration for individual, it comes as follows

In [3]:
df.sort_values(by='No of tasks',ascending = False)

,Worker Number,Worker_id,No of tasks,Correlation
67,67,54896d76fc4e7e0f300000b4,659,-0.011211
51,51,523949ca3465040cdb0078ac,384,-0.076515
16,16,512dbf09c32206698e000518,271,0.438533
13,13,5119d07bed964935a800008b,244,0.055305
2,2,50531712cb022432f3000f27,195,-0.085539
1,1,503b89aacb02245d320001a4,136,0.063627
87,87,5678e1a8fc4e7e9e0b000086,133,0.045923
33,33,51e60f42863a99e7d5025708,124,-0.163825
49,49,5232d065863a99092101733d,121,0.023250
72,72,566fc9c6fc4e7e1195000d55,114,-0.020008


> The encouraginf side is, for the workers with maximum number of tasks, correlation is negative. That means accuracy decreases as work duration increases. 

## Converting all the accuracy values in binary 

In [4]:
worker =pd.read_csv('worker.csv')

In [5]:
worker.head()

,Unnamed: 0,task_id,worker_id,work_duration,accuracy,mousemove_distance,mouseclick_button
0,0,57a2997ee2013f0c5a5d9f33,501767d1cb022453170004dc,88.318,0.833333,19.722592,0.068966
1,1,57a2997ee2013f0c5a5d9f34,501767d1cb022453170004dc,34.392,1.000000,7.173393,0.376812
2,2,57a2997ee2013f0c5a5d9f35,501767d1cb022453170004dc,18.640,1.000000,22.958495,0.061224
3,3,57a2997ee2013f0c5a5d9f36,501767d1cb022453170004dc,15.841,0.833333,8.415718,0.093023
4,4,57a2997ee2013f0c5a5d9f37,501767d1cb022453170004dc,15.844,1.000000,2.868044,0.155556


In [6]:
worker = pd.DataFrame(worker[['task_id','worker_id','work_duration','accuracy','mousemove_distance','mouseclick_button']])

In [7]:
worker['accuracy'] = worker['accuracy'] * 6

In [8]:
worker['accuracy'] = worker['accuracy'].round()

In [9]:
worker['accuracy'].value_counts()

6.0    2626
5.0     923
3.0     476
0.0     475
4.0     227
1.0     218
2.0      55
Name: accuracy, dtype: int64

In [10]:
worker.loc[worker['accuracy']!=6, 'accuracy'] = 0

In [11]:
worker['accuracy'].value_counts()

6.0    2626
0.0    2374
Name: accuracy, dtype: int64

In [12]:
worker

,task_id,worker_id,work_duration,accuracy,mousemove_distance,mouseclick_button
0,57a2997ee2013f0c5a5d9f33,501767d1cb022453170004dc,88.318,0.0,19.722592,0.068966
1,57a2997ee2013f0c5a5d9f34,501767d1cb022453170004dc,34.392,6.0,7.173393,0.376812
2,57a2997ee2013f0c5a5d9f35,501767d1cb022453170004dc,18.640,6.0,22.958495,0.061224
3,57a2997ee2013f0c5a5d9f36,501767d1cb022453170004dc,15.841,0.0,8.415718,0.093023
4,57a2997ee2013f0c5a5d9f37,501767d1cb022453170004dc,15.844,6.0,2.868044,0.155556
5,57a2997ee2013f0c5a5d9f38,501767d1cb022453170004dc,9.907,0.0,6.811904,0.108108
6,57a2997ee2013f0c5a5d9f39,501767d1cb022453170004dc,18.598,0.0,5.422883,0.160000
7,57a2997ee2013f0c5a5d9f3a,501767d1cb022453170004dc,75.675,6.0,13.941594,0.047619
8,57a2997ee2013f0c5a5d9f3b,501767d1cb022453170004dc,17.629,0.0,7.259340,0.142857
9,57a2997ee2013f0c5a5d9f3c,501767d1cb022453170004dc,114.393,0.0,37.741768,0.417391


#### making matrix with correlation for all workers

In [13]:
workers_list = worker['worker_id'].unique()
temp_list=[]
worker_mat_binary = pd.DataFrame(columns = ['Worker Number','Worker_id','No of tasks','Correlation'])
for i,j in enumerate (workers_list):
    dataframe = worker[worker['worker_id']==str(j)]
    count = dataframe.count(axis=0)[0]
    core = dataframe['work_duration'].corr(dataframe['accuracy'])
    temp = pd.DataFrame([[i,j,count,core]], columns=['Worker Number','Worker_id','No of tasks','Correlation'])
    temp_list.append(temp)
worker_mat_binary = worker_mat_binary.append(temp_list)

C:\Users\Chinmay\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [14]:
worker_mat_binary.sort_values(by='No of tasks',ascending = False)

,Worker Number,Worker_id,No of tasks,Correlation
0,67.0,54896d76fc4e7e0f300000b4,659.0,-0.055394
0,51.0,523949ca3465040cdb0078ac,384.0,-0.105735
0,16.0,512dbf09c32206698e000518,271.0,0.309306
0,13.0,5119d07bed964935a800008b,244.0,-0.015082
0,2.0,50531712cb022432f3000f27,195.0,-0.200715
0,1.0,503b89aacb02245d320001a4,136.0,-0.022037
0,87.0,5678e1a8fc4e7e9e0b000086,133.0,-0.029428
0,33.0,51e60f42863a99e7d5025708,124.0,-0.204268
0,49.0,5232d065863a99092101733d,121.0,0.029540
0,72.0,566fc9c6fc4e7e1195000d55,114.0,-0.105204


> If we see the above matrix, most of the people have negative correlation with work duration to accuracy. And that is very logical. 

In [15]:
worker_mat_binary_count = worker_mat_binary

In [16]:
worker_mat_binary_count.loc[worker_mat_binary_count['Correlation'] < 0, 'Correlation'] = -1
worker_mat_binary_count.loc[worker_mat_binary_count['Correlation'] >= 0, 'Correlation'] = 1

In [17]:
worker_mat_binary_count['Correlation'].value_counts()

-1.0    47
 1.0    39
Name: Correlation, dtype: int64

> Out of all people who have done more than 100 tasks (12 such people) 84% (10) people have negative correlation with accuracy to work duration. That means as work duration increases, accuracy decreases